Load packages

In [ ]:
import os
import os.path as osp
import numpy as np
from PIL import Image
import SimpleITK as sitk
import pandas as pd
from radiomics import featureextractor
from mrmr import mrmr_classif
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LassoCV
import matplotlib.pyplot as plt

### Radiomics
Step 1: Convert images(png) to nrrd

In [ ]:
def conver_png_to_nrrd(source_floder, save_floder):
    '''
    : param source_floder: source floder path
    : param save_floder: save floder path

    convert png to nrrd
    '''
    for root, dirs, files in os.walk(source_floder, topdown=False):
        for name in dirs:
            if name == 'cdfi':
                continue
            os.makedirs(osp.join(root, name).replace(source_floder, save_floder), exist_ok=True)

    for root, dirs, files in os.walk(source_floder, topdown=False):
        for name in files:
            if 'cdfi' in root:
                continue
            file = osp.join(root, name)
            img = np.asarray(Image.open(file).convert('L'))
            img = np.expand_dims(img, axis=2)
            img = sitk.GetImageFromArray(img) 
            sitk.WriteImage(img, file.replace(source_floder, save_floder).replace('png', 'nrrd'))

In [ ]:
conver_png_to_nrrd('../data/origin', '../data/nrrd')

Step 2: extract radiomics features

In [ ]:
def radiomics_feature_extractor(img_path, roi_path, yaml_path, save_path):
    '''
    : param img_path: image floder path
    : param roi_path: roi floder path
    : param yaml_path: pyradiomics config file path
    : param save_path: save csv path
    '''
    img_list = [osp.join(img_path, i) for i in os.listdir(img_path)]
    roi_path = [osp.join(roi_path, i) for i in os.listdir(roi_path)]

    feature_df = pd.DataFrame()
    names = []
    labels = []

    for img, roi in zip(img_list, roi_path):
        extractor = featureextractor.RadiomicsFeatureExtractor(yaml_path)
        featureVector = extractor.execute(img, roi)
        feature_item = pd.DataFrame.from_dict([featureVector.values()])
        feature_item.columns = featureVector.keys()
        feature_df = pd.concat([feature_df, feature_item])
        name = osp.basename(img).split('.')[0]
        names.append(name)
        if 'po' in name:
            labels.append(1)
        else:
            labels.append(0)
    
    feature_df = feature_df.iloc[:, 37:] # delete first 37 columns
    feature_df.insert(0, "label", labels)
    feature_df.insert(0, "names", names)

    os.makedirs(osp.dirname(save_path), exist_ok=True)

    feature_df.to_csv(save_path, index=False)

In [ ]:
radiomics_feature_extractor('../data/nrrd/PC/bus', '../data/nrrd/PC/roi', 'config.yaml', 'form/radiomics/pc_feature.csv')
radiomics_feature_extractor('../data/nrrd/VC/bus', '../data/nrrd/VC/roi', 'config.yaml', 'form/radiomics/vc_feature.csv')
radiomics_feature_extractor('../data/nrrd/TC1/bus', '../data/nrrd/TC1/roi', 'config.yaml', 'form/radiomics/tc1_feature.csv')
radiomics_feature_extractor('../data/nrrd/TC2/bus', '../data/nrrd/TC2/roi', 'config.yaml', 'form/radiomics/tc2_feature.csv')

Step 3: Rad Score

In [ ]:
def min_redundancy_max_relevance(dataframe, feature_nums=30):
    '''
    params dataframe: dataframe of features
    '''
    # the columns of dataframe: names, radiomics_features...

    X = dataframe.iloc[:, 2:]
    y = dataframe.iloc[:, 1]
    feature_index = mrmr_classif(X=X, y=y, K=feature_nums)
    feature_index.insert(0, 'label')
    return feature_index

In [ ]:
def lasso_selection(dataframe, lasso_alphas=[-4, 0, 50]):
    # the columns of dataframe: names, labels, radiomics_features...
    
    X = dataframe.iloc[:, 2:]
    Y = dataframe['label']

    columns = X.columns
    X = StandardScaler().fit_transform(X)
    X = pd.DataFrame(X)

    X.columns = columns
    alphas = np.logspace(lasso_alphas[0], lasso_alphas[1], lasso_alphas[2])
    lasso_model = LassoCV(alphas=alphas, max_iter=1000000000).fit(X, Y)
    coef = pd.Series(lasso_model.coef_, index=columns)

    print('Lasso picked : {}'.format(sum(coef != 0)))

    feature_index = coef[coef != 0].index
    feature_coef = coef[coef != 0]

    bias = lasso_model.intercept_

    x_values = np.arange(len(feature_index))
    y_values = coef[coef != 0]
    values_sort = sorted(zip(y_values.keys(), y_values.values), key=lambda x:x[1])
    feature_index, y_values = zip(*values_sort)

    
    coefs = lasso_model.path(X, Y, alphas=alphas, max_iter=100000)[1].T
    MSEs = lasso_model.mse_path_
    MSEs_mean = np.apply_along_axis(np.mean, 1, MSEs)
    MSEs_std = np.apply_along_axis(np.std, 1, MSEs)

    

    # lasso特征权重图
    fontsize = 12
    fontdict = {
        'family' : 'sans-serif',
        'fontsize': fontsize
    }
    lw = 2
    fig = plt.figure(1, figsize=(20,15), dpi=900)
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.3, hspace=0.2)
    fig.tight_layout()
    grid = plt.GridSpec(2,4)
    ax = plt.subplot(grid[1, 1:3])

    ax.barh(
        x_values,    
        y_values, 
        color='lightblue', 
        edgecolor='black', 
        alpha=0.8, 
        yerr=0.0001
    )

    ax.set_yticks(
        x_values, 
        feature_index, 
        # rotation='90',
        ha='right', 
        va='top',
        fontsize=fontsize
    )
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize) 
    ax.set_xlabel("weight", fontdict=fontdict)  # 横轴名称
    ax.set_ylabel("feature", fontdict=fontdict)  # 纵轴名称
    ax.spines['bottom'].set_linewidth(lw)
    ax.spines['left'].set_linewidth(lw)
    ax.spines['top'].set_linewidth(lw)
    ax.spines['right'].set_linewidth(lw)

    ax = plt.subplot(grid[0, 0:2])
    ax.spines['bottom'].set_linewidth(lw)
    ax.spines['left'].set_linewidth(lw)
    ax.spines['top'].set_linewidth(lw)
    ax.spines['right'].set_linewidth(lw)

    plt.errorbar(
        lasso_model.alphas_, 
        MSEs_mean, 
        yerr=MSEs_std,  # y误差范围
        fmt="o",        # 数据点标记
        ms=3,           # 数据点大小
        mfc="r",        # 数据点颜色
        mec="r",        # 数据点边缘颜色
        ecolor="lightblue",     # 误差棒颜色
        elinewidth=2,   # 误差棒线宽
        capsize=4,      # 误差棒边界线长度
        capthick=1      # 误差棒边界线厚度
    )  
    plt.semilogx()
    plt.axvline(lasso_model.alpha_, color='black', ls="--")
    plt.xlabel('Lambda', fontdict=fontdict)
    plt.ylabel('MSE', fontdict=fontdict)
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize) 


    ax = plt.subplot(grid[0,2:])
    ax.spines['bottom'].set_linewidth(lw)
    ax.spines['left'].set_linewidth(lw)
    ax.spines['top'].set_linewidth(lw)
    ax.spines['right'].set_linewidth(lw)
    plt.semilogx(lasso_model.alphas_, coefs, '-')
    plt.axvline(lasso_model.alpha_, color='black', ls="--")
    plt.xlabel('Lambda', fontdict=fontdict)
    plt.ylabel('Coefficients', fontdict=fontdict)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12) 
    # plt.show()
    plt.savefig('plot/lasso.svg', format='svg', dpi=900)


    feature_index = list(feature_index)
    feature_index.insert(0,'label')
    return feature_index, feature_coef, bias

In [ ]:
def feature_selection(dataframe, mrmr_feature_nums, lasso_alphas):
    # mrmr
    mrmr_feature_index = min_redundancy_max_relevance(dataframe, mrmr_feature_nums)
    dataframe = dataframe[mrmr_feature_index]

    # lasso
    lasso_feature_index, lasso_feature_coef, lasso_bias = lasso_selection(dataframe, lasso_alphas)

    return lasso_feature_index, lasso_feature_coef, lasso_bias


In [ ]:
def compute_rad_score(dataframe, features_index, coef, bias, save_path):
    '''
    计算lasso选出的特征乘以及对应的权重
    '''
    names = dataframe['names']
    dataframe = dataframe[features_index]
    colunms = ['rad_score']
    X = dataframe.iloc[:, 1:]
    X = StandardScaler().fit_transform(X)  # 将数值标准化
    label = dataframe.iloc[:,0]
    w = coef.to_numpy()

    score = ((w * X).sum(axis=1))
    score = score + bias

    rad_score = pd.DataFrame(score)
    rad_score.columns = colunms
    label = pd.DataFrame(label)

    rad_score.insert(0, 'label', label)
    rad_score.insert(0, 'names', names)

    os.makedirs(osp.dirname(save_path), exist_ok=True)

    rad_score.to_csv(osp.join(save_path), index=None)

In [ ]:
os.makedirs('plot', exist_ok=True)

In [ ]:
radiomics_pc = 'form/radiomics/pc_feature.csv'
radiomics_vc = 'form/radiomics/vc_feature.csv'
radiomics_tc1 = 'form/radiomics/tc1_feature.csv'
radiomics_tc2 = 'form/radiomics/tc2_feature.csv'

pc_rad_score_save_path = 'form/radiomics/pc.csv'
vc_rad_score_save_path = 'form/radiomics/vc.csv'
tc1_rad_score_save_path = 'form/radiomics/tc1.csv'
tc2_rad_score_save_path = 'form/radiomics/tc2.csv'

pc_df = pd.read_csv(radiomics_pc)
vc_df = pd.read_csv(radiomics_vc)
tc1_df = pd.read_csv(radiomics_tc1)
tc2_df = pd.read_csv(radiomics_tc2)

mrmr_feature_nums = 30
lasso_alphas = [-4, 0, 50]

lasso_feature_index, lasso_feature_coef, lasso_bias = feature_selection(pc_df, mrmr_feature_nums, lasso_alphas)
compute_rad_score(pc_df, lasso_feature_index, lasso_feature_coef, lasso_bias, pc_rad_score_save_path)
compute_rad_score(vc_df, lasso_feature_index, lasso_feature_coef, lasso_bias, vc_rad_score_save_path)
compute_rad_score(tc1_df, lasso_feature_index, lasso_feature_coef, lasso_bias, tc1_rad_score_save_path)
compute_rad_score(tc2_df, lasso_feature_index, lasso_feature_coef, lasso_bias, tc2_rad_score_save_path)

### Get Deep Score(code in code/deeplearning)

### Merge dataframe

In [ ]:
names = ['pc.csv', 'vc.csv', 'tc1.csv', 'tc2.csv']
dirs = ['clinical', 'radiomics', 'deeplearning']
save_path = 'form/merge'
os.makedirs(save_path, exist_ok=True)

for name in names:
    tmp_df = None
    for d in dirs:
        df = pd.read_csv(osp.join('form', d, name))
        if d == 'radiomics':
            df = df.drop('label', axis=1)
        if tmp_df is None:
            tmp_df = df
        else:
            tmp_df = pd.merge(tmp_df, df, on='names')
    tmp_df.to_csv(osp.join(save_path, name), index=None)

### Z-Score

In [ ]:
def zscore(name):
    df = pd.read_csv('form/merge/{}'.format(name))
    sub = StandardScaler().fit_transform(df.iloc[:, 2:])
    sub = pd.DataFrame(sub)
    df.iloc[:, 2:] = sub
    df.to_csv('form/merge/{}_zscore.csv'.format(name.split('.')[0]), index=None)

for name in ['pc.csv', 'vc.csv', 'tc1.csv', 'tc2.csv']:
    zscore(name)

### Clinical T test

In [ ]:
from scipy.stats import ttest_ind, levene
import pandas as pd

In [ ]:
def T_test_two(df1, df2):
    for colName in df1.columns[1:]:
        # 证明方差齐性
        if levene(df1[colName], df2[colName])[1] > 0.05:
            #如果大于0.05就是有方差齐性,用T检验
            print("{} : {}".format(colName, ttest_ind(df1[colName], df2[colName])[1]))
        else:
            #大于0.05无方差齐性，就讲equal_val=False 不用T检验用其他检验
            print("{} : {}".format(colName, ttest_ind(df1[colName], df2[colName], equal_var=False)[1]))

In [ ]:
def  T_test(data):
    '''
    :param data:数据
    :return: T检验筛选后的特征
    '''
    index_T = ['label']  #label不参与T检验
    data_a = data[:][data['label'] == 0]  # 数据正例
    data_b = data[:][data['label'] == 1]  # 数据负例
    print("pos mean", data_a.mean(axis=0))
    print("neg mean", data_b.mean(axis=0))
    print("pos std",data_a.std(axis=0))
    print("neg std", data_b.std(axis=0))
    for colName in data.columns[1:]:
        if levene(data_a[colName], data_b[colName])[1] > 0.05:
            if ttest_ind(data_a[colName], data_b[colName])[1] < 0.05:
                # print(colName)
                print(ttest_ind(data_a[colName], data_b[colName])[1])
                index_T.append(colName)
        else:
            if ttest_ind(data_a[colName], data_b[colName], equal_var=False)[1] < 0.05:
                # print(colName)
                print(ttest_ind(data_a[colName], data_b[colName], equal_var=False)[1])
                index_T.append(colName)
    return index_T

In [ ]:
clinical_pc = pd.read_csv('form/clinical/pc.csv')
clinical_vc = pd.read_csv('form/clinical/vc.csv')
clinical_tc1 = pd.read_csv('form/clinical/tc1.csv')
clinical_tc2 = pd.read_csv('form/clinical/tc2.csv')

In [ ]:
print('='*50)
print('PC VC')
T_test_two(clinical_pc, clinical_vc)
print('='*50)
print('PC TC1')
T_test_two(clinical_pc, clinical_tc1)
print('='*50)
print('PC TC2')
T_test_two(clinical_pc, clinical_tc2)
print('='*50)

In [ ]:
print('='*50)
print('PC')
T_test(clinical_pc)
print('='*50)
print('VC')
T_test(clinical_vc)
print('='*50)
print('TC1')
T_test(clinical_tc1)
print('='*50)
print('TC2')
T_test(clinical_tc2)
print('='*50)

### nomogram R(code/nomogram.ipynb)

### Plot

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import roc_curve, auc, confusion_matrix
import seaborn as sns

Get model predict prob from R

In [ ]:
PC = pd.read_csv('form/prob/pc.csv')
VC = pd.read_csv('form/prob/vc.csv')
TC1 = pd.read_csv('form/prob/tc1.csv')
TC2 = pd.read_csv('form/prob/tc2.csv')

PC['label'] = [1 if 'po' in i else 0 for i in PC['names']]
VC['label'] = [1 if 'po' in i else 0 for i in VC['names']]
TC1['label'] = [1 if 'po' in i else 0 for i in TC1['names']]
TC2['label'] = [1 if 'po' in i else 0 for i in TC2['names']]

DLRN_PC = PC['DLRN_PC']
Clinical_PC = PC['Clinical_PC']
Radiomics_PC = PC['Radiomics_PC']
DL_PC = PC['DL_PC']

DLRN_VC = VC['DLRN_VC']
Clinical_VC = VC['Clinical_VC']
Radiomics_VC = VC['Radiomics_VC']
DL_VC = VC['DL_VC']

DLRN_TC1 = TC1['DLRN_TC1']
Clinical_TC1 = TC1['Clinical_TC1']
Radiomics_TC1 = TC1['Radiomics_TC1']
DL_TC1 = TC1['DL_TC1']

DLRN_TC2 = TC2['DLRN_TC2']
Clinical_TC2 = TC2['Clinical_TC2']
Radiomics_TC2 = TC2['Radiomics_TC2']
DL_TC2 = TC2['DL_TC2']

ROC

In [ ]:
def plot_ROC(ax, label, DLRN_pred, Clinical_pred, Radiomics_pred, DL_pred, lw, fontdict, flag=False):
    # DLRN
    fpr, tpr, thread = roc_curve(label, DLRN_pred)
    roc_auc = auc(fpr, tpr)

    plt.plot(fpr, tpr, color='#EE2E22',lw=lw, label='DLRN (area = %0.3f)' % roc_auc)
    # Clinical
    fpr, tpr, thread = roc_curve(label, Clinical_pred)
    roc_auc = auc(fpr, tpr)

    plt.plot(fpr, tpr, color='#01539E',lw=lw, label='CM (area = %0.3f)' % roc_auc)
    # Radiomics
    fpr, tpr, thread = roc_curve(label, Radiomics_pred)
    roc_auc = auc(fpr, tpr)

    plt.plot(fpr, tpr, color='#E0C003',lw=lw, label='RM (area = %0.3f)' % roc_auc)
    # DL
    fpr, tpr, thread = roc_curve(label, DL_pred)
    roc_auc = auc(fpr, tpr)

    plt.plot(fpr, tpr, color='#006F3C',lw=lw, label='DM (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='gray', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    ax.spines['bottom'].set_linewidth(lw)
    ax.spines['left'].set_linewidth(lw)
    ax.spines['top'].set_linewidth(lw)
    ax.spines['right'].set_linewidth(lw)
    # for line in ax.lines:
    #     line.set_linewidth(lw)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12) 
    plt.xlabel('False Positive Rate', fontdict=fontdict)
    if flag:
        plt.ylabel('True Positive Rate', fontdict=fontdict, labelpad=10)
    else:
        plt.ylabel('', fontdict=fontdict, labelpad=10)
    # plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right", fontsize=10)

DCA

In [ ]:
def calculate_net_benefit_model(thresh_group, y_pred_score, y_label):
    net_benefit_model = np.array([])
    for thresh in thresh_group:
        y_pred_label = y_pred_score > thresh
        tn, fp, fn, tp = confusion_matrix(y_label, y_pred_label).ravel()
        n = len(y_label)
        net_benefit = (tp / n) - (fp / n) * (thresh / (1 - thresh))
        net_benefit_model = np.append(net_benefit_model, net_benefit)
    return net_benefit_model

def calculate_net_benefit_all(thresh_group, y_label):
    net_benefit_all = np.array([])
    tn, fp, fn, tp = confusion_matrix(y_label, y_label).ravel()
    total = tp + tn
    for thresh in thresh_group:
        net_benefit = (tp / total) - (tn / total) * (thresh / (1 - thresh))
        net_benefit_all = np.append(net_benefit_all, net_benefit)
    return net_benefit_all

def plot_DCA(ax, thresh_group, model_dlrn, model_clinic, model_radiomics, model_dl, net_benefit_all, fontdict,lw, flag=False):
    #Plot"#EE2E22","#01539E","#E0C003","#006F3C"
    ax.plot(thresh_group, model_dlrn, color = '#EE2E22', label = 'DLRN')
    ax.plot(thresh_group, model_clinic, color = '#01539E', label = 'Clinic')
    ax.plot(thresh_group, model_radiomics, color = '#E0C003', label = 'Radiomics')
    ax.plot(thresh_group, model_dl, color = '#006F3C', label = 'DL')
    ax.plot(thresh_group, net_benefit_all, color = 'gray',label = 'Treat all')
    ax.plot((0, 1), (0, 0), color='gray', lw=lw, linestyle='--', label = 'Treat none')
    
    ax.set_xlim(0,1)
    ax.set_xlabel(
        xlabel = 'Threshold Probability', 
        fontdict= fontdict
        )
    # plt.ylabel(ylabel='Net Benefit', labelpad=20, fontdict= fontdict)
    if flag:
        ax.set_ylabel(
            ylabel = 'Net Benefit', 
            labelpad=0,
            fontdict= fontdict
            )
    else:
        ax.set_ylabel(
            ylabel = '', 
            labelpad=0,
            fontdict= fontdict
            )
    # ax.grid('major')
    # ax.spines['right'].set_color((0.8, 0.8, 0.8))
    # ax.spines['top'].set_color((0.8, 0.8, 0.8))
    # ax.spines['right'].set_color((0.8, 0.8, 0.8))
    # ax.spines['top'].set_color((0.8, 0.8, 0.8))

    # for line in ax.lines:
    #     line.set_linewidth(lw)
    ax.spines['bottom'].set_linewidth(lw)
    ax.spines['left'].set_linewidth(lw)
    ax.spines['top'].set_linewidth(lw)
    ax.spines['right'].set_linewidth(lw)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12) 
    ax.legend(loc = 'upper right', fontsize=10)
    ax.set_ylim([-0.2,0.62])

    return None

Calibration Curve

In [ ]:
def plot_calibration_curve(ax, data, DLRN_pred, Clinical_pred, Radiomics_pred, DL_pred, fontdict, lw, n_bins=10, title='text', flag=False):
    y1, x1 = calibration_curve(data['label'], DLRN_pred, n_bins=n_bins, strategy='quantile')
    y2, x2 = calibration_curve(data['label'], Clinical_pred, n_bins=n_bins, strategy='quantile')
    y3, x3 = calibration_curve(data['label'], Radiomics_pred, n_bins=n_bins, strategy='quantile')
    y4, x4 = calibration_curve(data['label'], DL_pred, n_bins=n_bins, strategy='quantile')

    # fig = plt.figure(figsize=(8, 6))
    # ax = fig.subplots()
    plt.plot([0, 1], [0, 1], linestyle='--',color = 'gray')
    plt.plot(x1, y1, marker='.', label='DLRN', color = '#EE2E22')
    plt.plot(x2, y2, marker='.', label='CM',color = '#01539E')
    plt.plot(x3, y3, marker='.', label='RM',color = '#E0C003')
    plt.plot(x4, y4, marker='.', label='DM',color = '#006F3C')
    # plt.legend(fontsize=15)
    plt.xlabel("Predicted Value", fontdict=fontdict)
    if flag:
        plt.ylabel("Fraction of Positives", fontdict=fontdict, labelpad=10)
    else:
        plt.ylabel("", fontdict=fontdict, labelpad=10)
    # plt.text(0.45, 1.1, title, fontdict={'fontsize':18})
    plt.title(label=title, fontdict={'fontsize':18})
    ax.spines['bottom'].set_linewidth(lw)
    ax.spines['left'].set_linewidth(lw)
    ax.spines['top'].set_linewidth(lw)
    ax.spines['right'].set_linewidth(lw)
    plt.legend(loc='best', fontsize=10)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12) 
    #plt.savefig('calibrate.svg')

In [ ]:
# fig = plt.figure(1, figsize=(40,20), dpi=900)
fig = plt.figure(1, figsize=(22,15), dpi=900)

fontdict={
    'family' : 'sans-serif',
    'fontsize': 13
}
lw = 2

# 校准曲线
ax = plt.subplot(341)
plot_calibration_curve(ax, PC, DLRN_PC, Clinical_PC, Radiomics_PC, DL_PC, fontdict, lw, title='PC', flag=True)
ax = plt.subplot(342)
plot_calibration_curve(ax, VC, DLRN_VC, Clinical_VC, Radiomics_VC, DL_VC, fontdict, lw, title='VC')
ax = plt.subplot(343)
plot_calibration_curve(ax, TC1, DLRN_TC1, Clinical_TC1, Radiomics_TC1, DL_TC1, fontdict, lw, title='TC1')
ax = plt.subplot(344)
plot_calibration_curve(ax, TC2, DLRN_TC2, Clinical_TC2, Radiomics_TC2, DL_TC2, fontdict, lw, title='TC2')

# PC ROC
ax = plt.subplot(345)
plot_ROC(ax, PC['label'], DLRN_PC, Clinical_PC, Radiomics_PC, DL_PC, lw=lw, fontdict=fontdict, flag=True)
# VC ROC
ax = plt.subplot(346)
plot_ROC(ax, VC['label'],  DLRN_VC, Clinical_VC, Radiomics_VC, DL_VC, lw=lw, fontdict=fontdict)
# TC1 ROC
ax = plt.subplot(347)
plot_ROC(ax, TC1['label'], DLRN_TC1, Clinical_TC1, Radiomics_TC1, DL_TC1, lw=lw, fontdict=fontdict)
# TC2 ROC
ax = plt.subplot(348)
plot_ROC(ax, TC2['label'],  DLRN_TC2, Clinical_TC2, Radiomics_TC2, DL_TC2, lw=lw, fontdict=fontdict)




# PC DCA
ax = plt.subplot(349)
thresh_group = np.arange(0,1,0.02)
net_benefit_DLRN_tra = calculate_net_benefit_model(thresh_group, DLRN_PC, PC['label'])
net_benefit_Clinical_tra = calculate_net_benefit_model(thresh_group, Clinical_PC, PC['label'])
net_benefit_Radiomics_tra = calculate_net_benefit_model(thresh_group, Radiomics_PC, PC['label'])
net_benefit_DL_tra = calculate_net_benefit_model(thresh_group, DL_PC, PC['label'])
net_benefit_all_tra = calculate_net_benefit_all(thresh_group, PC['label'])
plot_DCA(ax, thresh_group, net_benefit_DLRN_tra,net_benefit_Clinical_tra, net_benefit_Radiomics_tra, net_benefit_DL_tra, net_benefit_all_tra, fontdict=fontdict,lw=lw, flag=True)

# VC DCA
ax = plt.subplot(3, 4, 10)
net_benefit_DLRN_val = calculate_net_benefit_model(thresh_group, DLRN_VC, VC['label'])
net_benefit_Clinical_val = calculate_net_benefit_model(thresh_group, Clinical_VC, VC['label'])
net_benefit_Radiomics_val = calculate_net_benefit_model(thresh_group, Radiomics_VC, VC['label'])
net_benefit_DL_val = calculate_net_benefit_model(thresh_group, DL_VC, VC['label'])
net_benefit_all_val = calculate_net_benefit_all(thresh_group, VC['label'])
plot_DCA(ax, thresh_group, net_benefit_DLRN_val, net_benefit_Clinical_val, net_benefit_Radiomics_val, net_benefit_DL_val, net_benefit_all_val, fontdict=fontdict,lw=lw)

# TC1 DCA
ax = plt.subplot(3, 4, 11)
thresh_group = np.arange(0,1,0.02)
net_benefit_DLRN_07 = calculate_net_benefit_model(thresh_group, DLRN_TC1, TC1['label'])
net_benefit_Clinical_07 = calculate_net_benefit_model(thresh_group, Clinical_TC1, TC1['label'])
net_benefit_Radiomics_07 = calculate_net_benefit_model(thresh_group, Radiomics_TC1, TC1['label'])
net_benefit_DL_07 = calculate_net_benefit_model(thresh_group, DL_TC1, TC1['label'])
net_benefit_all_07 = calculate_net_benefit_all(thresh_group, TC1['label'])
plot_DCA(ax, thresh_group, net_benefit_DLRN_07,net_benefit_Clinical_07, net_benefit_Radiomics_07, net_benefit_DL_07, net_benefit_all_07, fontdict=fontdict,lw=lw)

# TC2 DCA
ax = plt.subplot(3, 4, 12)
net_benefit_DLRN_15 = calculate_net_benefit_model(thresh_group, DL_TC2, TC2['label'])
net_benefit_Clinical_15 = calculate_net_benefit_model(thresh_group, Clinical_TC2, TC2['label'])
net_benefit_Radiomics_15 = calculate_net_benefit_model(thresh_group, Radiomics_TC2, TC2['label'])
net_benefit_DL_15 = calculate_net_benefit_model(thresh_group, DL_TC2, TC2['label'])
net_benefit_all_15 = calculate_net_benefit_all(thresh_group, TC2['label'])
plot_DCA(ax, thresh_group, net_benefit_DLRN_15, net_benefit_Clinical_15, net_benefit_Radiomics_15, net_benefit_DL_15, net_benefit_all_15, fontdict=fontdict,lw=lw)


fig.savefig('plot/Calibrate_ROC_DCA.svg')
plt.show()

Nomgram Points and Confusion Matrix

In [ ]:
PC_df = pd.read_csv('form/nom_score/pc.csv')
VC_df = pd.read_csv('form/nom_score/vc.csv')
TC1_df = pd.read_csv('form/nom_score/tc1.csv')
TC2_df = pd.read_csv('form/nom_score/tc2.csv')

In [ ]:
def plot_bar_cfm(ax, pred, label, threshhold = 0.5, flag=False):
    pred_neg = np.where(pred < threshhold, 1, 0)
    pred_pos = np.where(pred >= threshhold, 1, 0)

    TP = np.sum(pred_pos * label)
    FP = np.sum(pred_pos * (1 - label))
    TN = np.sum(pred_neg * (1 - label))
    FN = np.sum(pred_neg * label)
    plt.bar(['low_probability', 'high_probability'], [TN+FN, TP+FP], width=0.5, color='#f3764a', label='positive')
    plt.bar(['low_probability', 'high_probability'], [TN, FP], width=0.5, color='#01539E', label='negative')
    high = round(max(TN+FN, TP+FP)*1.2)
    plt.xticks(fontsize=12) 
    plt.yticks(fontsize=12) 
    plt.plot([0,0],[TN+FN,high], color='black')
    plt.plot([1,1],[TP+FP,high], color='black')
    plt.plot([0,1],[high,high], color='black')
    plt.text(0.3, high+10, 'P<0.0001',fontdict=fontdict)
    plt.ylim(0, high+100)
    plt.legend(loc='upper right', fontsize=10)
    lw=2
    ax.spines['bottom'].set_linewidth(lw)
    ax.spines['left'].set_linewidth(lw)
    ax.spines['top'].set_linewidth(lw)
    ax.spines['right'].set_linewidth(lw)
    if flag:
        plt.ylabel('Number of case', fontdict=fontdict)
    else:
        plt.ylabel('')

In [ ]:
def plot_box(ax, df, flag=False, title=''):
    ax = sns.boxplot(x='label', y='pred', data=df, width=0.4, linewidth=1.5, color='#ff6100')
    ax = sns.stripplot(
        data=df, x="label", y="pred",
        alpha=.25, zorder=1, jitter=0.15, color='black')
    bwith = 2
    ax.spines['bottom'].set_linewidth(bwith)
    ax.spines['left'].set_linewidth(bwith)
    ax.spines['top'].set_linewidth(bwith)
    ax.spines['right'].set_linewidth(bwith)
    plt.xticks([0, 1], ['benign', 'malignant'], fontsize=12) 
    plt.yticks(fontsize=12) 
    plt.title(label=title, fontdict={'fontsize':18})
    for line in plt.gca().lines:
        line.set_linewidth(2)
    # plt.xlabel('Malignancy of Breast Cancer', fontdict=fontdict)
    plt.xlabel('', fontdict=fontdict)
    if flag:
        plt.ylabel('DLRN Points', fontdict=fontdict)
    else:
        plt.ylabel('')


In [ ]:

fig = plt.figure(2, figsize=(20,10))

grid = plt.GridSpec(5,4,wspace=0.5,hspace=0.5)

# plt.subplots_adjust(left=None,bottom=None,right=None,top=None,wspace=0.5,hspace=0.5)
ax = plt.subplot(grid[:3,0])
plot_box(ax, PC_df, True, title='PC')
ax = plt.subplot(grid[:3,1])
plot_box(ax, VC_df, title='VC')
ax = plt.subplot(grid[:3,2])
plot_box(ax, TC1_df, title='TC1')
ax = plt.subplot(grid[:3,3])
plot_box(ax, TC2_df, title='TC2')
ax = plt.subplot(grid[3:,0])
plot_bar_cfm(ax, DLRN_PC, PC['label'], flag=True)
ax = plt.subplot(grid[3:,1])
plot_bar_cfm(ax, DLRN_VC, VC['label'])
ax = plt.subplot(grid[3:,2])
plot_bar_cfm(ax, DLRN_TC1, TC1['label'])
ax = plt.subplot(grid[3:,3])
plot_bar_cfm(ax, DLRN_TC2, TC2['label'])
plt.subplots_adjust(left=None, bottom=None, right=None, top=None,
                wspace=None, hspace=2)
fig.savefig('plot/confusion_matrix.svg')


nomogram in python

In [ ]:
from matplotlib import ticker
major = 10
minor = 2.5
fig = plt.figure(figsize=(8, 8))



def plt_nomogram(rows, column, current, starting, ending, major, minor, name, fontdict, site=[], labelpad=110):
    ax = plt.subplot(rows, column, current)
    ax.yaxis.set_major_locator(ticker.NullLocator())
    ax.spines['right'].set_color('none')
    ax.spines['left'].set_color('none')
    ax.spines['bottom'].set_color('none')
    ax.xaxis.set_major_locator(ticker.MultipleLocator(major))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(minor))
    # ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))
    # ax.xaxis.set_minor_formatter(ticker.FormatStrFormatter('%0.1f'))
    ax.xaxis.set_ticks_position("top")
    ax.set_xticks(np.around(np.arange(0, ending-starting+major, major), 2), labels=np.around(np.arange(starting, ending+major, major), 2))
    # print(np.arange(0, ending-starting+major, major))
    # print(np.arange(starting, ending+major, major))
    # ax.set_xlim(starting, ending)
    # plt.xticks(np.arange(0, ending-starting+major, major), labels=np.arange(starting, ending+major, major))

    ax.set_ylabel("{}".format(name), loc='bottom', fontdict=fontdict, fontname='Monospace', color='black', rotation=0, labelpad=labelpad)
    ax.set_position(site)



rows = 120
column = 1
X0 = 0.2
X1  = 83/200.*0.7+0.2
# 设置y的标签与x轴的间距
labelpad = 1000
# 修改site[x,y,w,h] x:subplot与当前左边间距 y：subplot与当前上边间距 w:subplot的宽 h：subplot的高

# [x0,x1,x2,x3]参数含义：
# x1:范围（0-1），根据有多少横轴进行等比例划分
# x2：分数：以plt——1做为100分的标准：0.7对应着100分，超过100分的按100分的长度画
# x3：忘了，应该是h：subplot的高
#
fontdict={
    'family' : 'sans-serif',
    'fontsize': 10
}

#总分
plt_nomogram(rows, column, 1, 0, 100, 10, 2.5, "Points", fontdict, [X0,0.9,0.7,0.001])
#最小值最大值
plt_nomogram(rows, column, 2, 17, 82, 13, 6.5, "Age", fontdict, [X0,0.8,0.7*0.48, 0.001])
plt_nomogram(rows, column, 3, 0.3, 5.4, 3.6,1.8, "Tumor long", fontdict, [X0, 0.7, 0.7*0.16, 0.001])
plt_nomogram(rows, column, 4, 0.3, 4.2, 3.6, 1.8, "Tumor short", fontdict, [X0,0.6,0.7*0.13,0.001])
plt_nomogram(rows, column, 5, -0.2, 1.5, 0.5, 0.25, "Rad score", fontdict, [X0,0.5,0.7*0.30,0.001])
plt_nomogram(rows, column, 6, -10.3, 6.2, 2, 1, "DL score", fontdict, [X0,0.4,0.7,0.001])

plt_nomogram(rows, column, 7, 0, 200, 20, 10, "Total points", fontdict, [X0,0.3,0.7,0.001])

# 分类结果
# plt_nomogram(rows, column, 8, 0, 100, 4, 1, "Risk", [X1, 0.2, 0.7*(129-83)/200.,0.001], labelpad=270)
ax = plt.subplot(rows, column, 8)
ax.yaxis.set_major_locator(ticker.NullLocator())
ax.spines['right'].set_color('none')
ax.spines['left'].set_color('none')
ax.spines['bottom'].set_color('none')
ax.xaxis.set_major_locator(ticker.MultipleLocator(major))
# ax.xaxis.set_minor_locator(ticker.MultipleLocator(minor))
ax.xaxis.set_ticks_position("top")
ax.set_xticks([0, 12, 23, 34, 46], labels=[0.01, 0.1, 0.5, 0.9, 0.99])
ax.set_ylabel("{}".format('Risk'), loc='bottom', fontdict=fontdict, fontname='Monospace', color='black', rotation=0, labelpad=275)
ax.set_position([X1, 0.2, 0.7*(129-83)/200.,0.001])


plt.show()
#
plt.savefig('plot/nom.svg',format='svg',dpi=1800)